In [1]:
import pandas as pd

c:\Users\Dell\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
storm = pd.read_parquet('../data/storm_2020_2022.parquet')
eaglei = pd.read_parquet('../data/eaglei_grouped_2020_2022.parquet')

In [3]:
storm.head(-1)

,start_datetime_eastern,end_datetime_eastern,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE_IMPUTED,MAGNITUDE_TYPE_IMPUTED
360974,2020-07-22 15:16:00,2020-07-22 15:16:00,0.0,FLORIDA,12,Thunderstorm Wind,C,65,JEFFERSON,TAE,0,0,0,0,3000.0,0.0,Law Enforcement,50.000000,EG
360975,2020-07-22 14:09:00,2020-07-22 14:09:00,0.0,FLORIDA,12,Thunderstorm Wind,C,67,LAFAYETTE,TAE,0,0,0,0,0.0,0.0,Law Enforcement,50.000000,EG
360976,2020-07-22 15:26:00,2020-07-22 15:26:00,0.0,FLORIDA,12,Thunderstorm Wind,C,65,JEFFERSON,TAE,0,0,0,0,3000.0,0.0,Law Enforcement,50.000000,EG
360977,2020-07-02 19:34:00,2020-07-02 19:34:00,0.0,KANSAS,20,Thunderstorm Wind,C,167,RUSSELL,ICT,0,0,0,0,0.0,0.0,ASOS,55.000000,MG
360978,2020-07-22 14:35:00,2020-07-22 14:35:00,0.0,FLORIDA,12,Thunderstorm Wind,C,123,TAYLOR,TAE,0,0,0,0,0.0,0.0,Law Enforcement,50.000000,EG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598983,2022-12-31 22:00:00,2023-01-03 21:00:00,71.0,UTAH,49,Heavy Snow,Z,109,BEAR LAKE AND BEAR RIVER VALLEY,SLC,0,0,0,0,0.0,0.0,COOP Observer,66.328418,MG
598984,2022-12-31 22:00:00,2023-01-03 21:00:00,71.0,UTAH,49,Heavy Snow,Z,108,WASATCH BACK,SLC,0,0,0,0,0.0,0.0,COOP Observer,66.328418,MG
598985,2022-12-31 22:00:00,2023-01-03 21:00:00,71.0,UTAH,49,Heavy Snow,Z,110,WASATCH MOUNTAINS I-80 NORTH,SLC,0,0,0,0,0.0,0.0,SNOTEL,66.328418,MG
598990,2022-12-31 22:00:00,2023-01-03 21:00:00,71.0,UTAH,49,Heavy Snow,Z,125,SOUTHERN MOUNTAINS,SLC,0,0,0,0,0.0,0.0,SNOTEL,66.328418,MG


In [4]:
eaglei.head(-1)

,state,customers,datetime
0,Alabama,36.0,2020-01-01 00:00:00
1,Alabama,30.0,2020-01-01 00:15:00
2,Alabama,28.0,2020-01-01 00:30:00
3,Alabama,39.0,2020-01-01 00:45:00
4,Alabama,34.0,2020-01-01 01:00:00
...,...,...,...
5228682,Wyoming,3.0,2022-12-31 22:30:00
5228683,Wyoming,4.0,2022-12-31 22:45:00
5228684,Wyoming,5.0,2022-12-31 23:00:00
5228685,Wyoming,4.0,2022-12-31 23:15:00


In [6]:
import pandas as pd

# Standardize state names to uppercase
storm['STATE'] = storm['STATE'].str.upper()
eaglei['state'] = eaglei['state'].str.upper()

# Convert datetime columns to datetime objects
storm['start_datetime_eastern'] = pd.to_datetime(storm['start_datetime_eastern'])
storm['end_datetime_eastern'] = pd.to_datetime(storm['end_datetime_eastern'])
eaglei['datetime'] = pd.to_datetime(eaglei['datetime'])

merged_list = []

# Process each state separately
for st in eaglei['state'].unique():
    outages_state = eaglei[eaglei['state'] == st].copy()
    storms_state = storm[storm['STATE'] == st].copy()
    
    if storms_state.empty:
        continue
    
    # Sort by time so merge_asof works correctly
    outages_state.sort_values('datetime', inplace=True)
    storms_state.sort_values('start_datetime_eastern', inplace=True)
    
    # Use merge_asof to get the most recent storm event (within a tolerance)
    tolerance = pd.Timedelta(hours=1)  # Adjust as needed
    merged_state = pd.merge_asof(outages_state, storms_state,
                                 left_on='datetime',
                                 right_on='start_datetime_eastern',
                                 direction='backward',
                                 tolerance=tolerance)
    
    # Filter to ensure the outage time falls before the storm event ended
    merged_state = merged_state[merged_state['datetime'] <= merged_state['end_datetime_eastern']]
    
    merged_list.append(merged_state)

merged = pd.concat(merged_list, ignore_index=True)
print(merged.head())


     state  customers            datetime start_datetime_eastern  \
0  ALABAMA      895.0 2020-01-04 03:00:00    2020-01-04 03:00:00   
1  ALABAMA      964.0 2020-01-04 03:15:00    2020-01-04 03:00:00   
2  ALABAMA      957.0 2020-01-04 03:30:00    2020-01-04 03:00:00   
3  ALABAMA      806.0 2020-01-04 03:45:00    2020-01-04 03:00:00   
4  ALABAMA     1029.0 2020-01-04 04:00:00    2020-01-04 03:00:00   

  end_datetime_eastern  duration_hours    STATE  STATE_FIPS   EVENT_TYPE  \
0  2020-01-04 06:00:00             3.0  ALABAMA         1.0  Strong Wind   
1  2020-01-04 06:00:00             3.0  ALABAMA         1.0  Strong Wind   
2  2020-01-04 06:00:00             3.0  ALABAMA         1.0  Strong Wind   
3  2020-01-04 06:00:00             3.0  ALABAMA         1.0  Strong Wind   
4  2020-01-04 06:00:00             3.0  ALABAMA         1.0  Strong Wind   

  CZ_TYPE  ...  WFO INJURIES_DIRECT INJURIES_INDIRECT  DEATHS_DIRECT  \
0       Z  ...  HUN             0.0               0.0         

In [8]:
merged

,state,customers,datetime,start_datetime_eastern,end_datetime_eastern,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,WFO,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE_IMPUTED,MAGNITUDE_TYPE_IMPUTED
0,ALABAMA,895.0,2020-01-04 03:00:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.000000,EG
1,ALABAMA,964.0,2020-01-04 03:15:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.000000,EG
2,ALABAMA,957.0,2020-01-04 03:30:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.000000,EG
3,ALABAMA,806.0,2020-01-04 03:45:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.000000,EG
4,ALABAMA,1029.0,2020-01-04 04:00:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.000000,EG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140181,WYOMING,4.0,2022-12-31 22:45:00,2022-12-31 22:00:00,2023-01-01 01:00:00,3.0,WYOMING,56.0,Winter Storm,Z,...,RIW,0.0,0.0,0.0,0.0,0.0,0.0,Park/Forest Service,51.741000,MG
140182,WYOMING,5.0,2022-12-31 23:00:00,2022-12-31 23:00:00,2023-01-02 16:00:00,41.0,WYOMING,56.0,Winter Storm,Z,...,RIW,0.0,0.0,0.0,0.0,0.0,0.0,Department of Highways,52.574381,MG
140183,WYOMING,4.0,2022-12-31 23:15:00,2022-12-31 23:00:00,2023-01-02 16:00:00,41.0,WYOMING,56.0,Winter Storm,Z,...,RIW,0.0,0.0,0.0,0.0,0.0,0.0,Department of Highways,52.574381,MG
140184,WYOMING,4.0,2022-12-31 23:30:00,2022-12-31 23:00:00,2023-01-02 16:00:00,41.0,WYOMING,56.0,Winter Storm,Z,...,RIW,0.0,0.0,0.0,0.0,0.0,0.0,Department of Highways,52.574381,MG


In [9]:
# Drop the 'state' column
merged = merged.drop(columns=['state'])

# Rename columns
merged = merged.rename(columns={
    'customers': 'people_affected',
    'datetime': 'power_outage_datetime',
    'start_datetime_eastern': 'storm_start_datetime_est',
    'end_datetime_eastern': 'storm_end_datetime_est'
})

# Display the first few rows to confirm changes
merged.head()

,people_affected,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,...,WFO,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE_IMPUTED,MAGNITUDE_TYPE_IMPUTED
0,895.0,2020-01-04 03:00:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,7.0,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.0,EG
1,964.0,2020-01-04 03:15:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,7.0,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.0,EG
2,957.0,2020-01-04 03:30:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,7.0,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.0,EG
3,806.0,2020-01-04 03:45:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,7.0,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.0,EG
4,1029.0,2020-01-04 04:00:00,2020-01-04 03:00:00,2020-01-04 06:00:00,3.0,ALABAMA,1.0,Strong Wind,Z,7.0,...,HUN,0.0,0.0,0.0,0.0,65000.0,0.0,Emergency Manager,35.0,EG


In [10]:
# Export to parquet
merged.to_parquet('../data/storm_and_outage_merged_2020_2022.parquet')